In [29]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from pprint import pprint as pp
import csv
from pathlib import Path
import seaborn as sns
from itertools import product
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline 

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.metrics import homogeneity_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import MiniBatchKMeans, DBSCAN

import gensim
from gensim import corpora

In [3]:
user = pd.read_csv('user_spec_단위.csv')
loan = pd.read_csv("loan_result_단위.csv")

In [4]:
user.columns =['application_id', 'user_id', 'gender', 'label_income_type',
       'label_employment_type', 'label_houseown_type', 'label_purpose',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
       'existing_loan_cnt', 'Age', 'credit_grade', 'yearly_income',
       'existing_loan_amt', 'desired_amount', 'work_day']

In [5]:
loan.columns = ['application_id', 'loanapply_insert_time', 'bank_id', 'product_id',
       'loan_rate', 'is_applied', 'loan_limit']

In [6]:
loan_notnull = loan.loc[loan['is_applied'].notnull()]
loan_notnull['is_applied'].value_counts()

0.0    9709675
1.0     554711
Name: is_applied, dtype: int64

In [7]:
loan_null = loan.loc[loan['is_applied'].isnull()]

In [8]:
# user_spec 과 train inner 조인
loan_user = pd.merge(loan_notnull, user,on='application_id', how='inner')

In [9]:
loan_user['is_applied'].value_counts()

0.0    1122473
1.0      43928
Name: is_applied, dtype: int64

In [10]:
loan_user.columns

Index(['application_id', 'loanapply_insert_time', 'bank_id', 'product_id',
       'loan_rate', 'is_applied', 'loan_limit', 'user_id', 'gender',
       'label_income_type', 'label_employment_type', 'label_houseown_type',
       'label_purpose', 'personal_rehabilitation_yn',
       'personal_rehabilitation_complete_yn', 'existing_loan_cnt', 'Age',
       'credit_grade', 'yearly_income', 'existing_loan_amt', 'desired_amount',
       'work_day'],
      dtype='object')

In [11]:
loan_user = loan_user[['application_id', 'user_id','bank_id', 'product_id', 'gender', 'Age',
       'loanapply_insert_time', 'loan_limit', 'loan_rate',  
       'credit_grade', 'yearly_income', 'label_income_type',
       'label_employment_type', 'label_houseown_type', 'label_purpose',
       'desired_amount', 'personal_rehabilitation_yn','personal_rehabilitation_complete_yn',
        'existing_loan_cnt', 'existing_loan_amt',
       'work_day','is_applied']]

In [12]:
loan_user = loan_user.drop(['loanapply_insert_time', 'application_id', 'user_id','product_id', 'bank_id'], axis = 1)

In [13]:
loan_user.columns

Index(['gender', 'Age', 'loan_limit', 'loan_rate', 'credit_grade',
       'yearly_income', 'label_income_type', 'label_employment_type',
       'label_houseown_type', 'label_purpose', 'desired_amount',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
       'existing_loan_cnt', 'existing_loan_amt', 'work_day', 'is_applied'],
      dtype='object')

In [14]:
dataset = loan_user

In [15]:
df = dataset

In [16]:
def prep_data(df: pd.DataFrame) -> (np.ndarray, np.ndarray):
    X = df.iloc[:, :-1].values
    y = df.is_applied.values
    return X, y

In [17]:
df.isnull().sum()

gender                                 0
Age                                    0
loan_limit                             0
loan_rate                              0
credit_grade                           0
yearly_income                          0
label_income_type                      0
label_employment_type                  0
label_houseown_type                    0
label_purpose                          0
desired_amount                         0
personal_rehabilitation_yn             0
personal_rehabilitation_complete_yn    0
existing_loan_cnt                      0
existing_loan_amt                      0
work_day                               0
is_applied                             0
dtype: int64

In [18]:
X, y = prep_data(df)

In [19]:
from imblearn.over_sampling import SMOTE

method = SMOTE()

X_resampled, y_resampled = method.fit_resample(X, y)

In [20]:
print(f'X shape: {X.shape}\ny shape: {y.shape}')

X shape: (1166401, 16)
y shape: (1166401,)


In [21]:
print(pd.value_counts(pd.Series(y)))
print("**************")
print("*****비교*****")
print("**************")
print(pd.value_counts(pd.Series(y_resampled)))

0.0    1122473
1.0      43928
dtype: int64
**************
*****비교*****
**************
0.0    1122473
1.0    1122473
dtype: int64


In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=0)

In [40]:
pip install h5py==2.10.0 

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install h5py==2.10.0 --user

Note: you may need to restart the kernel to use updated packages.


# Smote + Logistic

In [28]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train, y_train)

AttributeError: 'str' object has no attribute 'decode'

In [53]:
predicted = clf.predict(X_test)

In [54]:
from sklearn.metrics import classification_report
print('Classification report:\n', classification_report(y_test, predicted))

from sklearn.metrics import confusion_matrix
print('Confusion matrix:\n', confusion_matrix(y_test, predicted))

Classification report:
               precision    recall  f1-score   support

         0.0       0.75      0.73      0.74    196032
         1.0       0.73      0.75      0.74    194931

    accuracy                           0.74    390963
   macro avg       0.74      0.74      0.74    390963
weighted avg       0.74      0.74      0.74    390963

Confusion matrix:
 [[142210  53822]
 [ 48566 146365]]


In [55]:
####### f1 score 중간 확인 ######



from sklearn.metrics import f1_score

f1 = f1_score(y_test, predicted)
print('f1 score :', f1)

f1 score : 0.740867285216062


# SMOTE + Logistic 파이프라인

In [45]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import BorderlineSMOTE
resampling = BorderlineSMOTE()
model = LogisticRegression(solver='liblinear')

In [46]:
pipeline = Pipeline([('SMOTE', resampling), ('Logistic Regression', model)])

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [48]:
pipeline.fit(X_train, y_train) 
predicted = pipeline.predict(X_test)

In [49]:
print('Classifcation report:\n', classification_report(y_test, predicted))

print('Confusion matrix:\n', confusion_matrix(y_test, predicted))

Classifcation report:
               precision    recall  f1-score   support

         0.0       0.99      0.74      0.85    293173
         1.0       0.10      0.73      0.18     11386

    accuracy                           0.74    304559
   macro avg       0.54      0.74      0.51    304559
weighted avg       0.95      0.74      0.82    304559

Confusion matrix:
 [[218255  74918]
 [  3097   8289]]


-> 더 안좋아진?

# 랜포

In [56]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=5, n_estimators=20)

In [57]:
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=20, random_state=5)

In [58]:
predicted = model.predict(X_test)

In [59]:
from sklearn.metrics import accuracy_score
print(f'Accuracy Score:\n{accuracy_score(y_test, predicted):0.3f}')

Accuracy Score:
0.979


In [60]:
from sklearn.metrics import precision_recall_curve, average_precision_score

average_precision = average_precision_score(y_test, predicted)

precision_recall = precision_recall_curve(y_test, predicted)

In [61]:
probs = model.predict_proba(X_test)

In [62]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
print('ROC Score:')
print(roc_auc_score(y_test, probs[:,1]))
print('\nClassification Report:')
print(classification_report(y_test, predicted))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, predicted))

####### f1 score 중간 확인 ######



from sklearn.metrics import f1_score

f1 = f1_score(y_test, predicted)
print('f1 score :', f1)

ROC Score:
0.9959947938444498

Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98    196032
         1.0       0.98      0.98      0.98    194931

    accuracy                           0.98    390963
   macro avg       0.98      0.98      0.98    390963
weighted avg       0.98      0.98      0.98    390963


Confusion Matrix:
[[192424   3608]
 [  4573 190358]]
f1 score : 0.9789635816167264


In [26]:
average_precision = average_precision_score(y_test, predicted)
average_precision

0.12518322575221805

# 가중치 적용

In [ ]:
model = RandomForestClassifier(class_weight='balanced_subsample', random_state=5, n_estimators=100)

model.fit(X_train, y_train)

predicted = model.predict(X_test)
probs = model.predict_proba(X_test)

# Print the ROC curve, classification report and confusion matrix
print('ROC Score:')
print(roc_auc_score(y_test, probs[:,1]))
print('\nClassification Report:')
print(classification_report(y_test, predicted))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, predicted))

In [28]:
def get_model_results(X_train: np.ndarray, y_train: np.ndarray,
                      X_test: np.ndarray, y_test: np.ndarray, model):
  
    model.fit(X_train, y_train)

    predicted = model.predict(X_test)
    
    try:
        probs = model.predict_proba(X_test)
        print('ROC Score:')
        print(roc_auc_score(y_test, probs[:,1]))
    except AttributeError:
        pass

    print('\nClassification Report:')
    print(classification_report(y_test, predicted))
    print('\nConfusion Matrix:')
    print(confusion_matrix(y_test, predicted))

In [29]:
# 모델 옵션 변경해보기...

model = RandomForestClassifier(bootstrap=True,
                               class_weight={0:1, 1:12},
                               criterion='entropy',
                               max_depth=33,
                               min_samples_leaf=10, 
                               n_estimators=150,
                               n_jobs=-1,
                               random_state=5)

# Run the function get_model_results
get_model_results(X_train, y_train, X_test, y_test, model)

ROC Score:
0.9278698907394388

Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.95      0.97    193344
         1.0       0.32      0.55      0.41      7813

    accuracy                           0.94    201157
   macro avg       0.65      0.75      0.69    201157
weighted avg       0.96      0.94      0.95    201157


Confusion Matrix:
[[184439   8905]
 [  3546   4267]]


In [31]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, predicted)
print('f1 score :', f1)

f1 score : 0.25094922737306846


In [ ]:
# 파라미터 추천 받기1........ (오래 걸림)


param_grid = {'n_estimators': [1, 30, 33],
              'max_features': ['auto', 'log2'], 
              'max_depth': [4, 8, 10, 12],
              'criterion': ['gini', 'entropy']}

model = RandomForestClassifier(random_state=5)

CV_model = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='recall', n_jobs=-1)

CV_model.fit(X_train, y_train)
CV_model.best_params_

In [32]:
# 파라미터 추천 받기2........(오래 걸림)


from sklearn.model_selection import RandomizedSearchCV

model = RandomForestClassifier(random_state=5) #여기 괄호 안에 하이퍼파라미터들이 들어가는 곳인데, 여기선 할 필요 없다

param_grid = {
    'max_depth':[20, 25, 30, 33, 35], #7개
    'min_samples_leaf':[5,10,15,20]  #4개 --> 7x4 = 총 28개의 조합
}

n_iter = 5 #파라미터 검색 횟수; 여기선 5번
random_search = RandomizedSearchCV(model
                                  , param_distributions = param_grid
                                  , n_iter = n_iter
                                  , cv = 5)
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=5),
                   n_iter=5,
                   param_distributions={'max_depth': [20, 25, 30, 33, 35],
                                        'min_samples_leaf': [5, 10, 15, 20]})

In [ ]:
# 추천받은 파라미터들 넣기!!!!!!!!!!!!

model = RandomForestClassifier(class_weight={0:1,1:12},
                               criterion=,
                               max_depth=,
                               max_features='', 
                               min_samples_leaf=10,
                               n_estimators=30,
                               n_jobs=-1,
                               random_state=5)


get_model_results(X_train, y_train, X_test, y_test, model)

# 앙상블(하다 말았음)

In [ ]:
# 1. 로지스틱

model = LogisticRegression(class_weight={0:1, 1:15}, random_state=5, solver='liblinear')

# Get the model results
get_model_results(X_train, y_train, X_test, y_test, model)

In [ ]:
# Voting Classifier

clf1 = LogisticRegression(class_weight={0:1, 1:15},
                          random_state=5,
                          solver='liblinear')

clf2 = RandomForestClassifier(class_weight={0:1, 1:12}, 
                              criterion='gini', 
                              max_depth=8, 
                              max_features='log2',
                              min_samples_leaf=10, 
                              n_estimators=30, 
                              n_jobs=-1,
                              random_state=5)

clf3 = DecisionTreeClassifier(random_state=5,
                              class_weight="balanced")

ensemble_model = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('dt', clf3)], voting='hard')

get_model_results(X_train, y_train, X_test, y_test, ensemble_model)

In [ ]:
# 앙상블 최종

ensemble_model = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft', weights=[1, 4, 1], flatten_transform=True)

# Get results 
get_model_results(X_train, y_train, X_test, y_test, ensemble_model)

In [ ]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, predicted)
print('f1 score :', f1)